In [262]:
import numpy as np
import re

In [245]:
lambda_mic = []
Aud_invsec = []
E_lo_invcm = []
E_up_invcm = []
g_lo = []
g_up = []

In [246]:
import urllib.request
from pprint import pprint
from html_table_parser.parser import HTMLTableParser
import pandas as pd
def url_get_contents(url):
    req = urllib.request.Request(url=url)
    f=urllib.request.urlopen(req)
    return f.read()
xhtml = url_get_contents('https://www.gemini.edu/observing/resources/near-ir-resources/spectroscopy/important-h2-lines').decode('utf-8')
p=HTMLTableParser()
p.feed(xhtml)
data =(p.tables[0])

In [247]:
data = data[2:]

In [248]:
rows = []
for i in range(0, len(data)):
    if data[i] != ['']:
        row = [data[i][0]]
        for j in range(1, len(data[i])):
            if data[i][j] == '' or data[i][j] == '.':
                row.append(np.nan)
            else:
                row.append(float(data[i][j]))
        rows.append(row)
        print(row)
data = rows

['0-0 S(0)', 28.221, 354.35, 5.0, 510.0, 0.0003, 0.001, nan, nan, nan]
['0-0 S(1)', 17.035, 587.04, 21.0, 1015.0, 0.0048, 0.065, 0.003, 0.001, 0.001]
['0-0 S(2)', 12.279, 814.43, 9.0, 1682.0, 0.0276, 0.11, 0.008, 0.003, 0.002]
['0-0 S(3)', 9.6649, 1034.67, 33.0, 2504.0, 0.0984, 0.84, 0.091, 0.043, 0.03]
['0-0 S(4)', 8.0258, 1245.98, 13.0, 3474.0, 0.264, 0.4, 0.071, 0.04, 0.03]
['0-0 S(5)', 6.9091, 1447.36, 45.0, 4586.0, 0.588, 1.19, 0.36, 0.25, 0.2]
['0-0 S(6)', 6.1088, 1636.97, 17.0, 5829.0, 1.14, 0.29, 0.16, 0.13, 0.12]
['0-0 S(7)', 5.5115, 1814.4, 57.0, 7197.0, 2.0, 0.47, 0.53, 0.56, 0.57]
['0-0 S(8)', 5.0529, 1979.08, 21.0, 8677.0, 3.24, 0.07, 0.17, 0.22, 0.25]
['0-0 S(9)', 4.6947, 2139.06, 69.0, 10263.0, 4.9, 0.077, 0.4, 0.7, 0.92]
['0-0 S(10)', 4.4096, 2267.8, 25.0, 11940.0, 7.03, 0.008, 0.1, 0.22, 0.33]
['0-0 S(11)', 4.181, 2391.79, 81.0, 13703.0, 9.64, 0.006, 0.19, 0.57, 1.01]
['0-0 S(12)', 3.9947, 2503.3, 29.0, 15549.0, 12.7, 0.001, 0.037, 0.15, 0.31]
['0-0 S(13)', 3.8464, 259

In [249]:
dictionary = {'Lines':[vals[0] for vals in data],
              'Wavel, um':[vals[1] for vals in data],
              'Freq, cm^-1':[vals[2] for vals in data],
              'g_J, upper':[vals[3] for vals in data],
              'Temp, K':[vals[4] for vals in data],
              'A_vals, e-7 s':[vals[5] for vals in data],
              'LTE_vals':[vals[6] for vals in data]}

In [250]:
data = pd.DataFrame(data=dictionary)

In [251]:
s_deltaj = 2
q_deltaj = 0
o_deltaj = 1

s_indices = []
for i in range(0, len(data)):
    if 'S' in data['Lines'][i]:
        s_indices.append(i)
q_indices = []
for i in range(0, len(data)):
    if 'Q' in data['Lines'][i]:
        q_indices.append(i)
o_indices = []
for i in range(0, len(data)):
    if 'O' in data['Lines'][i]:
        o_indices.append(i)
        
# not sure if I need this??

In [252]:
h = 6.62607015e-34 #Joule seconds
kb = 1.380649e-23 # Joules/Kelvin
c = 3e8 #m/s
def energies_invcm(wavel, temp):
    E_up_J = kb*float(temp)
    delta_E = h*c/(float(wavel)*1e-6) #Joules
    E_lo_J = E_up_J - delta_E
    E_lower = E_lo_J/(h*c)
    E_upper = E_up_J/(h*c)
    return E_lower, E_upper

In [253]:
E_lo = []
E_up = []
for i in range(0, len(data)):
    E_lower, E_upper = energies_invcm(data['Wavel, um'][i], data['Temp, K'][i])
    E_lo.append(E_lower) # cm^-1
    E_up.append(E_upper) # cm^-1

In [281]:
def extract_number(string):
    characters = [x for x in string]
    return float(characters[6])
def g_lower(string):
    
    if 'S' in data['Lines'][i]:
        delta_j = 2
    elif 'Q' in data['Lines'][i]:
        delta_j = 0
    elif 'O' in data['Lines'][i]:
        delta_j =-2
        
    vib_state = extract_number(string)
    if (vib_state % 2) == 0:
        nuclear_weight = 1
    else:
        nuclear_weight = 3

In [282]:
lines = data['Lines']
g_up = data['g_J, upper']
for i in range(0, len(data)):
    g_lower(lines[i])

0.0 even
1.0 odd
2.0 even
3.0 odd
4.0 even
5.0 odd
6.0 even
7.0 odd
8.0 even
9.0 odd
1.0 odd
1.0 odd
1.0 odd
1.0 odd
1.0 odd
1.0 odd
1.0 odd
1.0 odd
1.0 odd
1.0 odd
2.0 even
2.0 even
2.0 even
2.0 even
0.0 even
1.0 odd
2.0 even
3.0 odd
4.0 even
5.0 odd
6.0 even
7.0 odd
8.0 even
9.0 odd
1.0 odd
1.0 odd
1.0 odd
2.0 even
3.0 odd
4.0 even
5.0 odd
6.0 even
7.0 odd
2.0 even
3.0 odd
4.0 even
5.0 odd
6.0 even
7.0 odd
8.0 even
0.0 even
1.0 odd
2.0 even
3.0 odd
4.0 even
5.0 odd
2.0 even
3.0 odd
4.0 even
5.0 odd
6.0 even
7.0 odd
0.0 even
1.0 odd
2.0 even
3.0 odd
4.0 even
5.0 odd
6.0 even
7.0 odd
2.0 even
3.0 odd
4.0 even
5.0 odd
6.0 even
3.0 odd
4.0 even
5.0 odd
5.0 odd
7.0 odd
0.0 even
1.0 odd
2.0 even
3.0 odd
4.0 even
5.0 odd
1.0 odd
2.0 even
3.0 odd
4.0 even
5.0 odd
2.0 even
3.0 odd
4.0 even
5.0 odd


In [277]:
extract_number(data['Lines'][51])

'1'